In [19]:
# univariate lstm example
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [20]:
# preparing independent and dependent features
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [24]:
# define input sequence
timeseries_data = [110, 125, 133, 146, 158, 172, 187, 196, 210]
# choose a number of time steps
n_steps = 3

x_data, y_data = prepare_data(timeseries_data, n_steps)
print({'x_data' : x_data, 'y_data' : y_data})

{'x_data': array([[110, 125, 133],
       [125, 133, 146],
       [133, 146, 158],
       [146, 158, 172],
       [158, 172, 187],
       [172, 187, 196]]), 'y_data': array([146, 158, 172, 187, 196, 210])}


In [27]:
n_feature = 1
x_data = x_data.reshape((x_data.shape[0], x_data.shape[1], n_feature))
print(x_data)

[[[110]
  [125]
  [133]]

 [[125]
  [133]
  [146]]

 [[133]
  [146]
  [158]]

 [[146]
  [158]
  [172]]

 [[158]
  [172]
  [187]]

 [[172]
  [187]
  [196]]]


In [56]:
def model_training(x_data, y_data, n_steps, n_feature):
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_feature)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    model.fit(x_data, y_data, epochs=300, verbose=1)
    return model


In [58]:
model = model_training(x_data, y_data, n_steps, n_feature)
print(model)

Epoch 1/300
1/1 [==============================] - 7s 7s/step - loss: 34643.8008
Epoch 2/300
1/1 [==============================] - 0s 17ms/step - loss: 33882.6953
Epoch 3/300
1/1 [==============================] - 0s 17ms/step - loss: 33053.0586
Epoch 4/300
1/1 [==============================] - 0s 16ms/step - loss: 32190.7344
Epoch 5/300
1/1 [==============================] - 0s 19ms/step - loss: 31378.9512
Epoch 6/300
1/1 [==============================] - 0s 21ms/step - loss: 30697.2402
Epoch 7/300
1/1 [==============================] - 0s 18ms/step - loss: 30151.5938
Epoch 8/300
1/1 [==============================] - 0s 16ms/step - loss: 29673.2676
Epoch 9/300
1/1 [==============================] - 0s 20ms/step - loss: 29219.4746
Epoch 10/300
1/1 [==============================] - 0s 20ms/step - loss: 28766.1621
Epoch 11/300
1/1 [==============================] - 0s 19ms/step - loss: 28290.4551
Epoch 12/300
1/1 [==============================] - 0s 20ms/step - loss: 27779.4590
Epo

In [59]:
def feature_values(x_input, feature_year, n_steps, n_feature):
    x_input = array(x_input)
    temp_input = list(x_input)
    feature_revenue = []

    current_year = 2025
    while current_year < feature_year:
        if len(temp_input) > 3:
            x_input = array(temp_input[1:])
            x_input = x_input.reshape((1, n_steps, n_feature))
            yhat = model.predict(x_input, verbose=0)
            temp_input.append(yhat[0][0])
            temp_input=temp_input[1:]
            feature_revenue.append(yhat[0][0])
            current_year += 1
        else:
            x_input = x_input.reshape((1, n_steps, n_feature))
            yhat = model.predict(x_input, verbose=0)
            temp_input.append(yhat[0][0])
            feature_revenue.append(yhat[0][0])
            current_year += 1

    return feature_revenue


In [66]:
x_input = [62,72,82]
predicted_values = feature_values(x_input, 2032, n_steps, n_feature)
print([round(val, 2) for val in predicted_values])

[78.64, 89.86, 95.94, 99.43, 111.14, 118.94, 128.85]
